# Exploring the IBL Data Pipeline

Here we will introduce some useful DataJoint tools and concepts to help you explore the IBL data pipeline. Before proceeding make sure that you have installed the [IBL python environment](../../02_installation.md) and set up your [Datajoint credentials](../../dj_docs/dj_credentials.md)



## A few definitions
First of all, let's define a few basic definitions:

- Table - collection of rows and columns that contain data
- Schema - a collection of related tables 
- Module - script where schema and associated tables are defined
- Pipeline - collection of schemas

Example nomenclature would be to say that we want to get data from the `Subject table` stored in the `ibl_subjects schema` which are together defined in the `subject module` in the `IBL pipeline`. 

In practice, `modules` are often referred to as schemas and so we would refer to the `subject module` as the `subject schema`

Let's now proceed by importing Datajoint and some schemas from the IBL pipeline

In [ ]:
import datajoint as dj
dj.config['display.limit'] = 5
from ibl_pipeline import reference, subject, action, acquisition, data, behavior, ephys
from ibl_pipeline.analyses import behavior as behavior_analyses

## 1. Browsing schemas
The IBL pipeline contains a number of different schemas. To browse which schemas are available, we can use the `dj.list_schemas()` command,

In [ ]:
dj.list_schemas()


### Major schemas include:   
Meta data from **Alyx**: `ibl_reference`, `ibl_subject`, `ibl_action`, `ibl_acquisition`, and `ibl_data`  
Imported data from **FlatIron**: `ibl_behavior` and `ibl_ephys`  
Computed analzyed results: `ibl_analyses_behavior`


## 2. Browsing tables in a schemas
We can see what tables are defined in a schema using the `dj.Diagram` command. For example, to see the tables defined in the `subject` schema we can type,

```
dj.Diagram()
```

<div class="alert alert-info">

Note

For more information about the what the different colours and line types mean, please refer to this more comprehensive [tutorial](https://github.com/int-brain-lab/IBL-pipeline/blob/master/notebooks/notebooks_tutorial/201909_code_camp/1-Explore%20IBL%20data%20pipeline%20with%20DataJoint.ipynb)
</div>

We can also use the following code snippet to list the tables that are defined in a schema

In [ ]:
from datajoint.user_tables import UserTable
import inspect


def list_tables(schema):
    for k in dir(schema):
        t = getattr(schema, k)
        if inspect.isclass(t) and issubclass(t, UserTable):
            print(k)

In [ ]:
list_tables(ephys)

## 3. Getting the detailed definition of a table
To find out details about a table, we can use the `describe` method

In [ ]:
subject.Subject.describe()

## 4. Browsing data in tables  - queries

Query all subjects

In [ ]:
subject.Subject()

### Restriction  `&`: filtering data

#### Restriction: Query one subject

In [ ]:
# restrict by string
subject.Subject & 'subject_nickname="ibl_witten_10"'

In [ ]:
# restrict by dictionary
from uuid import UUID
subject.Subject & {'subject_uuid': UUID('00c60db3-74c3-4ee2-9df9-2c84acf84e92')}

#### Restriction: Query all male subjects

In [ ]:
subject.Subject & {'sex': 'm'}

#### Restriction: Query subjects born after a date

In [ ]:
subject.Subject & 'subject_birth_date > "2019-01-01"'

#### Restriction: Query subjects within a range of dates

In [ ]:
subject.Subject & 'subject_birth_date between "2019-01-01" and "2019-04-01"'

#### Restriction: Query subjects on multiple attributes

In [ ]:
subject.Subject & 'subject_birth_date > "2019-01-01"' & 'sex="M"'

#### Restriction: Query subjects restricted by other tables

In [ ]:
# subjects with Ephys recording
subject.Subject & ephys.ProbeInsertion

In [ ]:
# subjects without Ephys recording
subject.Subject - ephys.ProbeInsertion

### Join `*`:  gather information from different tables

In [ ]:
subject.Subject * acquisition.Session

### Projection `.proj()`: focus on attributes of interest

In [ ]:
subject.Subject.proj()

In [ ]:
subject.Subject.proj('subject_birth_date', 'sex')

#### rename attribute with ***proj()***

In [ ]:
subject.Subject.proj('sex', dob='subject_birth_date')

#### perform simple computations with ***proj***

**Example 1: Get the date of a session**

In [ ]:
sessions_with_date = acquisition.Session.proj(session_date='date(session_start_time)')

In [ ]:
sessions_with_date

**Example 2: Get the age of the animal at each session**

In [ ]:
# First get the date of birth and the session date into the same query
q = subject.Subject * acquisition.Session

In [ ]:
# Then compute the age
q_with_age = q.proj(age='datediff(session_start_time, subject_birth_date)')
q_with_age

### Aggregation `.aggr()`: simple computation of one table against another table

**Example: How many sessions has each subject done so far?**

In [ ]:
subject.Subject.aggr(acquisition.Session, 'subject_nickname', n='count(*)')

## 5. Fetching data

### Fetch all fields: `fetch()`

In [ ]:
# fetch all data from a table
subjs = subject.Subject.fetch()
subjs[:5]

In [ ]:
subjs['subject_uuid'][:5]

In [ ]:
subjs['subject_birth_date'][:5]

In [ ]:
# fetch as a list of dictionaries
subjs_dict = subject.Subject.fetch(as_dict=True)
subjs_dict[:5]

In [ ]:
# fetch as pandas dataframe
subjs_df = subject.Subject.fetch(format='frame')
subjs_df[:5]

In [ ]:
# fetch the primary key
pk = subject.Subject.fetch('KEY')
pk[:5]

In [ ]:
# fetch specific attributes
dob, sex = subject.Subject.fetch('subject_birth_date', 'sex')
dob[:5]

In [ ]:
info = subject.Subject.fetch('subject_birth_date', 'sex', as_dict=True)
info[:5]

### Fetch data from only one entry: `fetch1`

In [ ]:
ibl_witten_10 = (subject.Subject & {'subject_nickname': 'ibl_witten_10'}).fetch1('KEY')

In [ ]:
ibl_witten_10

In [ ]:
IBL_10 = (subject.Subject & {'subject_nickname': 'IBL_10'}).fetch1()

In [ ]:
IBL_10